In [200]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [201]:
# Read in IMDb csv file
csv_file1 = "Resources/IMDb_movies.csv"
imdb_df = pd.read_csv(csv_file1)
imdb_df

/Users/granthicks/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,...,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,...,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,...,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,$ 2833,NaN,NaN,NaN


In [202]:
# Check the datatypes of each column:
imdb_df.dtypes

imdb_title_id             object
title                     object
original_title            object
year                      object
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

In [203]:
# Change the year datatype to int64
imdb_df.astype({'year': 'int64'})

ValueError: invalid literal for int() with base 10: 'TV Movie 2019'

In [204]:
# Remove the row with 'TV Movie 2019' as the year
new_imdb_df = imdb_df[imdb_df.year != 'TV Movie 2019']

# Now change the year column datatype to integer
new_imdb_df = new_imdb_df.astype({'year': 'int64'})
new_imdb_df.dtypes

imdb_title_id             object
title                     object
original_title            object
year                       int64
date_published            object
genre                     object
duration                   int64
country                   object
language                  object
director                  object
writer                    object
production_company        object
actors                    object
description               object
avg_vote                 float64
votes                      int64
budget                    object
usa_gross_income          object
worlwide_gross_income     object
metascore                float64
reviews_from_users       float64
reviews_from_critics     float64
dtype: object

In [205]:
# Filter the columns we want to have in our database
clean_imdb_df = new_imdb_df[['title', 'year', 'genre', 'duration', 'country', 'avg_vote', 'votes', ]].copy()

# Change avg_vote to imdb_rating
clean_imdb_df = clean_imdb_df.rename(columns={"avg_vote": "imdb_rating",})


# Reorder the column names into a more logical order
clean_imdb_df = clean_imdb_df[['title', 'year', 'country', 'duration', 'imdb_rating', 'votes', 'genre']]

# We only want to see movies that are made in the USA and that have sound (first movie with sound was in 1927 so after 1926)
clean_imdb_df = clean_imdb_df.loc[(clean_imdb_df['country'] == 'USA') & (clean_imdb_df['year'] > 1926)]

# Drop any duplicates
clean_imdb_df.drop_duplicates("title", inplace=True)
clean_imdb_df

,title,year,country,duration,imdb_rating,votes,genre
427,Il piccolo colonnello,1935,USA,81,7.1,1420,"Comedy, Family, Musical"
494,His First Flame,1927,USA,48,6.0,127,Comedy
535,L'allievo di West Point,1927,USA,95,6.4,1722,"Drama, Romance, Sport"
538,Almost Human,1927,USA,60,7.5,339,"Drama, Romance"
540,Annie Laurie,1927,USA,90,5.9,355,"Drama, Romance"
...,...,...,...,...,...,...,...
85829,Season of Love,2019,USA,105,7.5,228,"Comedy, Romance"
85830,Bulletproof 2,2020,USA,97,3.5,326,"Action, Comedy"
85837,VFW,2019,USA,92,6.1,4178,"Action, Crime, Horror"
85838,The Pilgrim's Progress,2019,USA,108,5.7,442,"Animation, Adventure, Family"


In [224]:
# We want only good movies which we are defining as above an 7.0 rating
good_imdb_df = clean_imdb_df.loc[(clean_imdb_df['imdb_rating'] >= 7.0)].sort_values(by='imdb_rating', ascending=False)
good_imdb_df

,title,year,country,duration,imdb_rating,votes,genre
58809,Hopeful Notes,2010,USA,94,9.7,669,Drama
54166,The Moving on Phase,2020,USA,85,9.5,172,Comedy
81109,Love in Kilnerry,2019,USA,100,9.3,1239,Comedy
28453,Le ali della libertà,1994,USA,142,9.3,2278845,Drama
52570,As I Am,2019,USA,62,9.3,1173,"Drama, Fantasy, Romance"
...,...,...,...,...,...,...,...
7651,Per ritrovarti,1953,USA,95,7.0,259,Drama
22409,A distanza ravvicinata,1986,USA,111,7.0,13871,"Crime, Drama"
59614,Missing Pieces,2012,USA,117,7.0,130,Drama
59706,Maleficent,2014,USA,97,7.0,336610,"Action, Adventure, Family"


In [232]:
#Read rotten tomatoes file into pandas df
csv_file2 = "Resources/rotten_tomatoes.csv"
rottentomatoes_df = pd.read_csv(csv_file2)
rottentomatoes_df.head()

,Rank,Title,RatingTomatometer,No. of Reviews,Genres
0,1,Black Panther (2018),97,444,action|adventure
1,2,Mad Max: Fury Road (2015),97,394,action|adventure
2,3,Wonder Woman (2017),93,410,action|adventure
3,4,Metropolis (1927),99,118,action|adventure
4,5,Coco (2017),97,308,action|adventure


In [233]:
#Rename the columns
rottentomatoes_df.rename(columns={'Rank': 'rank', 'Title':'title', 'RatingTomatometer': 'ratingtomatometer', 'No. of Reviews': 'no_of_reviews', 'Genres': 'genre'}, 
                 inplace=True)
rottentomatoes_df

,rank,title,ratingtomatometer,no_of_reviews,genre
0,1,Black Panther (2018),97,444,action|adventure
1,2,Mad Max: Fury Road (2015),97,394,action|adventure
2,3,Wonder Woman (2017),93,410,action|adventure
3,4,Metropolis (1927),99,118,action|adventure
4,5,Coco (2017),97,308,action|adventure
...,...,...,...,...,...
1585,70,Priest (2011),15,97,western
1586,71,American Outlaws (2001),14,103,western
1587,72,September Dawn (2007),15,54,western
1588,73,Jonah Hex (2010),12,147,western


In [234]:
# Extract the year from the title column
year = rottentomatoes_df['title'].str[-7:]
print(year)

year_clean = year.str[0:7]
year_clean

0        (2018)
1        (2015)
2        (2017)
3        (1927)
4        (2017)
         ...   
1585     (2011)
1586     (2001)
1587     (2007)
1588     (2010)
1589     (2001)
Name: title, Length: 1590, dtype: object


0        (2018)
1        (2015)
2        (2017)
3        (1927)
4        (2017)
         ...   
1585     (2011)
1586     (2001)
1587     (2007)
1588     (2010)
1589     (2001)
Name: title, Length: 1590, dtype: object

In [235]:
# Seperate the year from the title in the title column and make new title and year columns
rottentomatoes_df['title_clean'] = rottentomatoes_df.title.str.split('(', expand = True)[0]
rottentomatoes_df['year'] = year_clean
rottentomatoes_df.head()

,rank,title,ratingtomatometer,no_of_reviews,genre,title_clean,year
0,1,Black Panther (2018),97,444,action|adventure,Black Panther,(2018)
1,2,Mad Max: Fury Road (2015),97,394,action|adventure,Mad Max: Fury Road,(2015)
2,3,Wonder Woman (2017),93,410,action|adventure,Wonder Woman,(2017)
3,4,Metropolis (1927),99,118,action|adventure,Metropolis,(1927)
4,5,Coco (2017),97,308,action|adventure,Coco,(2017)


In [236]:
# Drop the old title column
clean_rottentomatoes_df = rottentomatoes_df.drop(columns=['title'])
# Rename the title column
clean_rottentomatoes_df.rename(columns={'title_clean':'title'}, 
                 inplace=True)
# Reorder the columns
clean_rottentomatoes_df = clean_rottentomatoes_df[['title', 'year', 'genre', 'rank', 'ratingtomatometer', 'no_of_reviews']]
clean_rottentomatoes_df.drop_duplicates("title", inplace=True)
clean_rottentomatoes_df

,title,year,genre,rank,ratingtomatometer,no_of_reviews
0,Black Panther,(2018),action|adventure,1,97,444
1,Mad Max: Fury Road,(2015),action|adventure,2,97,394
2,Wonder Woman,(2017),action|adventure,3,93,410
3,Metropolis,(1927),action|adventure,4,99,118
4,Coco,(2017),action|adventure,5,97,308
...,...,...,...,...,...,...
1585,Priest,(2011),western,70,15,97
1586,American Outlaws,(2001),western,71,14,103
1587,September Dawn,(2007),western,72,15,54
1588,Jonah Hex,(2010),western,73,12,147


In [237]:
# Only keep movies that have a rating tomatometer score of 70 or greater
good_rottentomatoes_df = clean_rottentomatoes_df.loc[clean_rottentomatoes_df['ratingtomatometer'] >= 70].sort_values(by='ratingtomatometer', ascending=False)
good_rottentomatoes_df

,title,year,genre,rank,ratingtomatometer,no_of_reviews
214,Battleship Potemkin,(1925),art_house|international,15,100,45
322,The Bride of Frankenstein,(1935),classics,23,100,43
105,Toy Story 2,(1999),animation,6,100,163
110,Pinocchio,(1940),animation,11,100,47
349,Anatomy of a Murder,(1959),classics,50,100,42
...,...,...,...,...,...,...
1441,Talladega Nights: The Ballad of Ricky Bobby,(2006),sports|fitness,42,71,184
1446,More Than a Game,(2009),sports|fitness,47,71,51
1502,Only Human,(2006),television,40,71,52
1503,Waking Sleeping Beauty,(2010),television,41,70,50


In [229]:
#add unique id column
good_rottentomatoes_df['movie_id'] = good_rottentomatoes_df.reset_index().index
good_rottentomatoes_df

,title,year,genre,rank,ratingtomatometer,no_of_reviews,movie_id
214,Battleship Potemkin,(1925),art_house|international,15,100,45,0
322,The Bride of Frankenstein,(1935),classics,23,100,43,1
105,Toy Story 2,(1999),animation,6,100,163,2
110,Pinocchio,(1940),animation,11,100,47,3
349,Anatomy of a Murder,(1959),classics,50,100,42,4
...,...,...,...,...,...,...,...
1441,Talladega Nights: The Ballad of Ricky Bobby,(2006),sports|fitness,42,71,184,881
1446,More Than a Game,(2009),sports|fitness,47,71,51,882
1502,Only Human,(2006),television,40,71,52,883
1503,Waking Sleeping Beauty,(2010),television,41,70,50,884


In [215]:
# import movielens movie data
csv_file3 = "resources/movie.csv"
movielens_movie_df = pd.read_csv(csv_file3)
movielens_movie_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [115]:
# import movielens rating data
csv_file4 = "resources/rating.csv"
movielens_rating_df = pd.read_csv(csv_file4)
movielens_rating_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [116]:
# get the avg rating for each movieId
movielens_rating_mean = movielens_rating_df.groupby(by='movieId').mean()
movielens_rating_mean.head()

,userId,rating
movieId,,
1,69282.396821,3.921240
2,69169.928202,3.211977
3,69072.079388,3.151040
4,69652.913280,2.861393
5,69113.475454,3.064592


In [117]:
# merge movielens df (movie and rating) by column movieId
movielens_df = movielens_movie_df.merge(movielens_rating_mean, how='inner', on="movieId")
movielens_df

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,69282.396821,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,69169.928202,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,69072.079388,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,69652.913280,2.861393
4,5,Father of the Bride Part II (1995),Comedy,69113.475454,3.064592
...,...,...,...,...,...
26739,131254,Kein Bund für's Leben (2007),Comedy,79570.000000,4.000000
26740,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.000000,4.000000
26741,131258,The Pirates (2014),Adventure,28906.000000,2.500000
26742,131260,Rentun Ruusu (2001),(no genres listed),65409.000000,3.000000


In [118]:
# drop userId column
movielens_df.drop(columns='userId', inplace=True)
movielens_df.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.861393
4,5,Father of the Bride Part II (1995),Comedy,3.064592


In [119]:
# get clean title column without the year
movielens_df['title_clean'] = movielens_df.title.str.split('(', expand=True)[0]
movielens_df

,movieId,title,genres,rating,title_clean
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.921240,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.211977,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,3.151040,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.861393,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,3.064592,Father of the Bride Part II
...,...,...,...,...,...
26739,131254,Kein Bund für's Leben (2007),Comedy,4.000000,Kein Bund für's Leben
26740,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,4.000000,"Feuer, Eis & Dosenbier"
26741,131258,The Pirates (2014),Adventure,2.500000,The Pirates
26742,131260,Rentun Ruusu (2001),(no genres listed),3.000000,Rentun Ruusu


In [120]:
# get year data
year = movielens_df['title'].str[-7:]
year.head()

0     (1995)
1     (1995)
2     (1995)
3     (1995)
4     (1995)
Name: title, dtype: object

In [199]:
# clean year data
year_clean = year.str[2:6]
year_clean.head()

0    2018
1    2015
2    2017
3    1927
4    2017
Name: title, dtype: object

In [122]:
# Put year in own column
movielens_df['year'] = year_clean
movielens_df

,movieId,title,genres,rating,title_clean,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.921240,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.211977,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,3.151040,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.861393,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,3.064592,Father of the Bride Part II,1995
...,...,...,...,...,...,...
26739,131254,Kein Bund für's Leben (2007),Comedy,4.000000,Kein Bund für's Leben,2007
26740,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,4.000000,"Feuer, Eis & Dosenbier",2002
26741,131258,The Pirates (2014),Adventure,2.500000,The Pirates,2014
26742,131260,Rentun Ruusu (2001),(no genres listed),3.000000,Rentun Ruusu,2001


In [123]:
# drop title column
movielens_df.drop(columns='title', inplace=True)
movielens_df.head()

,movieId,genres,rating,title_clean,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,3.921240,Toy Story,1995
1,2,Adventure|Children|Fantasy,3.211977,Jumanji,1995
2,3,Comedy|Romance,3.151040,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,2.861393,Waiting to Exhale,1995
4,5,Comedy,3.064592,Father of the Bride Part II,1995


In [124]:
# rename data columns
movielens_df = movielens_df.rename(columns={'movieId': 'movieid', 'rating': 'movielens_rating', 'title_clean': 'title', 'genres': 'genre'})
movielens_df.head()

,movieid,genre,movielens_rating,title,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,3.921240,Toy Story,1995
1,2,Adventure|Children|Fantasy,3.211977,Jumanji,1995
2,3,Comedy|Romance,3.151040,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,2.861393,Waiting to Exhale,1995
4,5,Comedy,3.064592,Father of the Bride Part II,1995


In [125]:
# Rearrange column order
movielens_df = movielens_df[['movieid','title','year','movielens_rating','genre']]
movielens_df

,movieid,title,year,movielens_rating,genre
0,1,Toy Story,1995,3.921240,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,3.211977,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,3.151040,Comedy|Romance
3,4,Waiting to Exhale,1995,2.861393,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,3.064592,Comedy
...,...,...,...,...,...
26739,131254,Kein Bund für's Leben,2007,4.000000,Comedy
26740,131256,"Feuer, Eis & Dosenbier",2002,4.000000,Comedy
26741,131258,The Pirates,2014,2.500000,Adventure
26742,131260,Rentun Ruusu,2001,3.000000,(no genres listed)


In [127]:
# filter out good movies or movies with rating >= 4.0
good_movielens_df = movielens_df.loc[movielens_df['movielens_rating'] >= 4.0]
good_movielens_df

,movieid,title,year,movielens_rating,genre
27,28,Persuasion,1995,4.057546,Drama|Romance
46,47,Seven,1995,4.053493,Mystery|Thriller
49,50,"Usual Suspects, The",1995,4.334372,Crime|Mystery|Thriller
81,82,Antonia's Line,1995,4.004925,Comedy|Drama
108,110,Braveheart,1995,4.042534,Action|Drama|War
...,...,...,...,...,...
26737,131250,No More School,2000,4.000000,Comedy
26738,131252,Forklift Driver Klaus: The First Day on the Job,2001,4.000000,Comedy|Horror
26739,131254,Kein Bund für's Leben,2007,4.000000,Comedy
26740,131256,"Feuer, Eis & Dosenbier",2002,4.000000,Comedy


In [246]:
# Extract the whitespaces at the end of titles in the rottentomatoes and movie lens dataframes
good_rottentomatoes_df.title = good_rottentomatoes_df.title.str.rstrip()
good_movielens_df.title = good_movielens_df.title.str.rstrip()

/Users/granthicks/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [251]:
# Put all of the finished dataframes in a list
all_dfs = [good_imdb_df, good_movielens_df, good_rottentomatoes_df]

# Combine all three dataframes together by title for the final complete df
from functools import reduce
combined_df = reduce(lambda left,right: pd.merge(left,right,on='title'), all_dfs)
combined_df

,title,year_x,country,duration,imdb_rating,votes,genre_x,movieid,year_y,movielens_rating,genre_y,year,genre,rank,ratingtomatometer,no_of_reviews
0,Schindler's List,1993,USA,195,8.9,1183248,"Biography, Drama, History",527,1993,4.310175,Drama|War,(1993),drama,98,97,93
1,Casablanca,1942,USA,102,8.5,509953,"Drama, Romance, War",912,1942,4.258327,Drama|Romance,(1942),classics,9,97,77
2,Apocalypse Now,1979,USA,147,8.4,591251,"Drama, Mystery, War",1208,1979,4.108900,Action|Drama|War,(1979),action|adventure,46,96,85
3,Django Unchained,2012,USA,165,8.4,1317856,"Drama, Western",99114,2012,4.037893,Action|Drama|Western,(2012),western,19,86,274
4,Taxi Driver,1976,USA,114,8.3,703264,"Crime, Drama",111,1976,4.110576,Crime|Drama|Thriller,(1976),drama,39,98,82
5,Up,2009,USA,96,8.2,911402,"Animation, Adventure, Comedy",68954,2009,4.038266,Adventure|Animation|Children|Drama,(2009),action|adventure,18,98,290
6,Chinatown,1974,USA,130,8.2,286798,"Drama, Mystery, Thriller",1252,1974,4.199673,Crime|Film-Noir|Mystery|Thriller,(1974),classics,47,98,64
7,L.A. Confidential,1997,USA,138,8.2,521530,"Crime, Drama, Mystery",1617,1997,4.083377,Crime|Film-Noir|Mystery|Thriller,(1997),drama,64,99,109
8,Blade Runner,1982,USA,117,8.1,677998,"Action, Sci-Fi, Thriller",541,1982,4.133706,Action|Sci-Fi|Thriller,(1982),science_fiction|fantasy,88,90,110
9,Manhattan,1979,USA,96,7.9,129186,"Comedy, Drama, Romance",1244,1979,4.071039,Comedy|Drama|Romance,(1979),comedy,99,95,62


In [252]:
# Clean up the columns we want in our complete good movies database:
combined_df = combined_df[['title', 'year', 'genre', 'duration', 'imdb_rating', 'movielens_rating', 'ratingtomatometer']]
combined_df

,title,year,genre,duration,imdb_rating,movielens_rating,ratingtomatometer
0,Schindler's List,(1993),drama,195,8.9,4.310175,97
1,Casablanca,(1942),classics,102,8.5,4.258327,97
2,Apocalypse Now,(1979),action|adventure,147,8.4,4.108900,96
3,Django Unchained,(2012),western,165,8.4,4.037893,86
4,Taxi Driver,(1976),drama,114,8.3,4.110576,98
5,Up,(2009),action|adventure,96,8.2,4.038266,98
6,Chinatown,(1974),classics,130,8.2,4.199673,98
7,L.A. Confidential,(1997),drama,138,8.2,4.083377,99
8,Blade Runner,(1982),science_fiction|fantasy,117,8.1,4.133706,90
9,Manhattan,(1979),comedy,96,7.9,4.071039,95


In [255]:
# Export combined good movies dataframe to csv:
combined_df.to_csv ('Resources/combined_df.csv', index = False, header=True)

Connect to a local database

In [258]:
rds_connection_string = "postgres:postgres@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

Check for tables

In [259]:
engine.table_names()

['imdb', 'movie_lens', 'rotten_tomatoes']

Use pandas to load csv converted DataFrame into database

In [260]:
# Moving IMDb dataframe into movies_db imdb table
good_imdb_df.to_sql(name='imdb', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "imdb_pkey"
DETAIL:  Key (title)=(Hopeful Notes) already exists.

[SQL: INSERT INTO imdb (title, year, country, duration, imdb_rating, votes, genre) VALUES (%(title)s, %(year)s, %(country)s, %(duration)s, %(imdb_rating)s, %(votes)s, %(genre)s)]
[parameters: ({'title': 'Hopeful Notes', 'year': 2010, 'country': 'USA', 'duration': 94, 'imdb_rating': 9.7, 'votes': 669, 'genre': 'Drama'}, {'title': 'The Moving on Phase', 'year': 2020, 'country': 'USA', 'duration': 85, 'imdb_rating': 9.5, 'votes': 172, 'genre': 'Comedy'}, {'title': 'Love in Kilnerry', 'year': 2019, 'country': 'USA', 'duration': 100, 'imdb_rating': 9.3, 'votes': 1239, 'genre': 'Comedy'}, {'title': 'Le ali della libertà', 'year': 1994, 'country': 'USA', 'duration': 142, 'imdb_rating': 9.3, 'votes': 2278845, 'genre': 'Drama'}, {'title': 'As I Am', 'year': 2019, 'country': 'USA', 'duration': 62, 'imdb_rating': 9.3, 'votes': 1173, 'ge

In [261]:
# Moving Rotten Tomatoes dataframe into movies_db rotten_tomatoes table
good_rottentomatoes_df.to_sql(name='rotten_tomatoes', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "genre" of relation "rotten_tomatoes" does not exist
LINE 1: INSERT INTO rotten_tomatoes (title, year, genre, rank, ratin...
                                                  ^

[SQL: INSERT INTO rotten_tomatoes (title, year, genre, rank, ratingtomatometer, no_of_reviews) VALUES (%(title)s, %(year)s, %(genre)s, %(rank)s, %(ratingtomatometer)s, %(no_of_reviews)s)]
[parameters: ({'title': 'Battleship Potemkin', 'year': ' (1925)', 'genre': 'art_house|international', 'rank': 15, 'ratingtomatometer': 100, 'no_of_reviews': 45}, {'title': 'The Bride of Frankenstein', 'year': ' (1935)', 'genre': 'classics', 'rank': 23, 'ratingtomatometer': 100, 'no_of_reviews': 43}, {'title': 'Toy Story 2', 'year': ' (1999)', 'genre': 'animation', 'rank': 6, 'ratingtomatometer': 100, 'no_of_reviews': 163}, {'title': 'Pinocchio', 'year': ' (1940)', 'genre': 'animation', 'rank': 11, 'ratingtomatometer': 100, 'no_of_reviews': 47}, {'title': 'Anatomy of a 

In [262]:
# Moving Movie Lens dataframe into movies_db movie_lens table
good_movielens_df.to_sql(name='movie_lens', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "genre" of relation "movie_lens" does not exist
LINE 1: ...vie_lens (movieid, title, year, movielens_rating, genre) VAL...
                                                             ^

[SQL: INSERT INTO movie_lens (movieid, title, year, movielens_rating, genre) VALUES (%(movieid)s, %(title)s, %(year)s, %(movielens_rating)s, %(genre)s)]
[parameters: ({'movieid': 28, 'title': 'Persuasion', 'year': '1995', 'movielens_rating': 4.057545973367152, 'genre': 'Drama|Romance'}, {'movieid': 47, 'title': 'Seven', 'year': '1995', 'movielens_rating': 4.053492566302111, 'genre': 'Mystery|Thriller'}, {'movieid': 50, 'title': 'Usual Suspects, The', 'year': '1995', 'movielens_rating': 4.334372207803259, 'genre': 'Crime|Mystery|Thriller'}, {'movieid': 82, 'title': "Antonia's Line", 'year': '1995', 'movielens_rating': 4.004925137903862, 'genre': 'Comedy|Drama'}, {'movieid': 110, 'title': 'Braveheart', 'year': '1995', 'movielens_rating': 4.042533

Confirm data has been added by querying the table

In [263]:
# Query the IMDb table
pd.read_sql_query('select * from imdb', con=engine).head()

,title,year,country,duration,imdb_rating,votes,genre
0,Hopeful Notes,2010,USA,94,10,669,Drama
1,The Moving on Phase,2020,USA,85,10,172,Comedy
2,Love in Kilnerry,2019,USA,100,9,1239,Comedy
3,Le ali della libertà,1994,USA,142,9,2278845,Drama
4,As I Am,2019,USA,62,9,1173,"Drama, Fantasy, Romance"


In [264]:
# Query the Rotten Tomatoes table
pd.read_sql_query('select * from rotten_tomatoes', con=engine).head()

,title,year,genres,rank,ratingtomatometer,no_of_reviews
0,Afghan Star,2009,television,2,100,60
1,The Maltese Falcon,1941,mystery|suspense,10,100,53
2,The Wages of Fear,1953,art_house|international,11,100,43
3,Open City,1946,art_house|international,12,100,42
4,Tokyo Story,1953,art_house|international,13,100,41


In [265]:
# Query the Movie Lens table
pd.read_sql_query('select * from movie_lens', con=engine).head()

,movieid,title,year,movielens_rating,genres
0,28,Persuasion,1995,4,Drama|Romance
1,47,Seven,1995,4,Mystery|Thriller
2,50,"Usual Suspects, The",1995,4,Crime|Mystery|Thriller
3,82,Antonia's Line,1995,4,Comedy|Drama
4,110,Braveheart,1995,4,Action|Drama|War
